# Example notebook

This is an example notebook that demonstrates how to use the bh24-literature-mining package to extract information from the literature.

In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from bh24_literature_mining.utils import load_biotools_pub

tools = load_biotools_pub("../biotoolspub/biotoolspub.tsv")

for index, row in tools.head().iterrows():
    name = row["name"]
    pubmedid = row["pubmedid"]
    link = row["link"]
    print(f"Name: {name}, PubMed ID: {pubmedid}, Link: {link}")

Name: SubtiWiki, PubMed ID: PMC8728116, Link: fulltexts/PMC8728116.xml
Name: MetaPathways, PubMed ID: PMC4595896, Link: fulltexts/PMC4595896.xml
Name: iPHoP, PubMed ID: PMC10155999, Link: fulltexts/PMC10155999.xml
Name: MetaPathways, PubMed ID: PMC3695837, Link: fulltexts/PMC3695837.xml
Name: MZmine, PubMed ID: PMC1187873, Link: fulltexts/PMC1187873.xml


In [12]:
from bh24_literature_mining.europepmc_api import EuropePMCClient 


client = EuropePMCClient()
# Call bio.tools query and get a list of Article objects
biotools_articles = client.search_mentions("PeptideProphet")
print("Bio.tools articles:", biotools_articles)
# Call cites query with a specific PubMed ID and get a list of Article objects
cites_articles = client.search_cites("32109013")
print("Cites articles:", cites_articles)

Bio.tools articles: [Article(id='38687997', title='Reinvestigating the Correctness of Decoy-Based False Discovery Rate Control in Proteomics Tandem Mass Spectrometry.', authorString='Freestone J, Noble WS, Keich U.', pubYear='2024', journalTitle='J Proteome Res', pubDate=None, doi='10.1021/acs.jproteome.3c00902', pmcid=None, pmid='38687997'), Article(id='PPR900266', title='Tesorai Search: Large pretrained model boosts identifications in mass spectrometry proteomics without the need for Percolator', authorString='Burq M, Stepec D, Restrepo J, Zbontar J, Urazbakhtin S, Crampton B, Tiwary S, Miao M, Cox J, Cimermancic P.', pubYear='2024', journalTitle='', pubDate=None, doi='10.1101/2024.08.19.606805', pmcid=None, pmid=None), Article(id='38252705', title='AttnPep: A Self-Attention-Based Deep Learning Method for Peptide Identification in Shotgun Proteomics.', authorString='Li Y, He Q, Guo H, Shuai SC, Cheng J, Liu L, Shuai J.', pubYear='2024', journalTitle='J Proteome Res', pubDate=None, do